In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from numpy import pi, random
from IPython.display import display, Latex
import ipywidgets as widgets

# Initialize CHSH Game Logic

def chsh_game(strategy):
    x, y = random.randint(0, 2), random.randint(0, 2)
    a, b = strategy(x, y)
    if (int(a) != int(b)) == (x & y):
        return 1
    return 0

# Implement CHSH Quantum Circuit Construction

def chsh_circuit(x, y):
    qr = QuantumRegister(2, "q")
    cr = ClassicalRegister(2, "c")
    qc = QuantumCircuit(qr, cr)
    qc.h(0)
    qc.cx(0, 1)
    qc.barrier()
    if x == 0:
        qc.ry(0, 0)
    else:
        qc.ry(-pi / 2, 0)
    qc.measure(0, 0)
    if y == 0:
        qc.ry(-pi / 4, 1)
    else:
        qc.ry(pi / 4, 1)
    qc.measure(1, 1)
    return qc

# Display the Four Possible Strategy Circuits

outs = [widgets.Output() for _ in range(4)]
coords = [(0,0), (0,1), (1,0), (1,1)]
for i, (x, y) in enumerate(coords):
    with outs[i]:
        display(Latex(f"Circuit for $(x,y) = ({x},{y})$"))
        display(chsh_circuit(x, y).draw(output="mpl"))
display(widgets.VBox([widgets.HBox([outs[0], outs[1]]), widgets.HBox([outs[2], outs[3]])]))

# Define Quantum and Classical Strategies

def quantum_strategy(x, y):
    sim = AerSimulator()
    job = sim.run(chsh_circuit(x, y), shots=1)
    result = job.result()
    stats = result.get_counts()
    bits = list(stats.keys())[0]
    return bits[1], bits[0]
def classical_strategy(x, y):
    a = x
    b = 0
    return a, b

# Simulate Games and Display Results

NumGames = 1000
TotalQuantum = 0
TotalClassical = 0

for _ in range(NumGames):
    TotalQuantum += chsh_game(quantum_strategy)
    TotalClassical += chsh_game(classical_strategy)

display(Latex(f"CHSH Game Results ({NumGames} Games)"))
display(Latex(f"Quantum Strategy Win Rate:{TotalQuantum/NumGames:.4f}"))
display(Latex(f"Classical Strategy Win Rate: {TotalClassical/NumGames:.4f}"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>